# Tratamento de dados: junção, combinação e reformatação

## Indexação hierárquica

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])

data

a  1    0.397078
   2    0.006979
   3    0.467798
b  1   -0.719114
   3    0.865098
c  1    0.018024
   2    0.615756
d  2   -0.340135
   3   -0.528446
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b']

1   -0.719114
3    0.865098
dtype: float64

In [5]:
data['b':'c']

b  1   -0.719114
   3    0.865098
c  1    0.018024
   2    0.615756
dtype: float64

In [6]:
data.loc[['b','d']]

b  1   -0.719114
   3    0.865098
d  2   -0.340135
   3   -0.528446
dtype: float64

In [7]:
data.loc[:,2]

a    0.006979
c    0.615756
d   -0.340135
dtype: float64

In [8]:
data.unstack()

,1,2,3
a,0.397078,0.006979,0.467798
b,-0.719114,NaN,0.865098
c,0.018024,0.615756,NaN
d,NaN,-0.340135,-0.528446


In [9]:
data.unstack().stack()

a  1    0.397078
   2    0.006979
   3    0.467798
b  1   -0.719114
   3    0.865098
c  1    0.018024
   2    0.615756
d  2   -0.340135
   3   -0.528446
dtype: float64

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),index=[['a','a','b','b'],[1,2,1,2]],columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1','key2']

frame.columns.names = ['state','color']

frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [13]:
pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']],names=['state','color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reorganizando  e ordenando níveis

In [14]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [15]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Estatísticas de resumo por nível

In [17]:
frame.sum(level='key2')

/tmp/ipykernel_328/2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [18]:
frame.sum(level='color',axis=1)

/tmp/ipykernel_328/1534194635.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color',axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexando com as colunas de um DataFrame

In [19]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],'d':[0,1,2,0,1,2,3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [20]:
frame2 = frame.set_index(['c','d'])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [21]:
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [22]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combinando e mesclando conjuntos de dados

### Junções no DataFrame no estilo de bancos de dados

In [23]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})

df2 = pd.DataFrame({'key':['a','b','b'],'data2':range(3)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [24]:
df2

,key,data2
0,a,0
1,b,1
2,b,2


In [25]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,0,2
2,b,1,1
3,b,1,2
4,b,6,1
5,b,6,2
6,a,2,0
7,a,4,0
8,a,5,0


In [26]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,0,2
2,b,1,1
3,b,1,2
4,b,6,1
5,b,6,2
6,a,2,0
7,a,4,0
8,a,5,0


In [27]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})

df4 = pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})

pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [28]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0,1.0
1,b,0,2.0
2,b,1,1.0
3,b,1,2.0
4,b,6,1.0
5,b,6,2.0
6,a,2,0.0
7,a,4,0.0
8,a,5,0.0
9,c,3,NaN


In [29]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'], 'data1':range(6)})

df2 = pd.DataFrame({'key':['a','b','a','b','d'], 'data2':range(5)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [30]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [31]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [32]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [33]:
left = pd.DataFrame({'key1':['foo','foo','bar'],'key2':['one','two','one'],'lval':[1,2,3]})

right = pd.DataFrame({'key1':['foo','foo','bar','bar'],'key2':['one','one','one','two'],'rval':[4,5,6,7]})

pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [34]:
pd.merge(left,right,on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [35]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [36]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'),indicator=True)

,key1,key2_left,lval,key2_right,rval,_merge
0,foo,one,1,one,4,both
1,foo,one,1,one,5,both
2,foo,two,2,one,4,both
3,foo,two,2,one,5,both
4,bar,one,3,one,6,both
5,bar,one,3,two,7,both


### Fazendo merge com base no índice

In [37]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})

right1 = pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [38]:
right1

,group_val
a,3.5
b,7.0


In [39]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [40]:
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [41]:
lefth = pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],'key2':[2000,2001,2002,2001,2002],'data':np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6,2)),index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],[2001,2000,2000,2000,2001,2002]],columns=['event1','event2'])

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [42]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [43]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [44]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [45]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],index=['a','c','e'],columns=['Ohio','Nevada'])

right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],index=['b','c','d','e'],columns=['Missouri','Alabama'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [46]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [47]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [48]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [49]:
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [50]:
another = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],index=['a','c','e','f'],columns=['New York','Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [51]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [52]:
left2.join([right2,another],how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenando ao longo de um eixo

In [53]:
arr = np.arange(12).reshape((3,4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [54]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [55]:
s1 = pd.Series([0,1],index=['a','b'])

s2 = pd.Series([2,3,4],index=['c','d','e'])

s3 = pd.Series([5,6],index=['f','g'])

pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [56]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [57]:
s4 = pd.concat([s1,s3])

s4

a    0
b    1
f    5
g    6
dtype: int64

In [58]:
pd.concat([s1,s3],axis=1)

,0,1
a,0.0,NaN
b,1.0,NaN
f,NaN,5.0
g,NaN,6.0


In [60]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,1


In [69]:
result = pd.concat([s1,s2,s3],keys=['one', 'two', 'three'])

result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [70]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [71]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [72]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])

df1

,one,two
a,0,1
b,2,3
c,4,5


In [73]:
df2

,three,four
a,5,6
c,7,8


In [74]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [75]:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a','b','c','d'])

df2 = pd.DataFrame(np.random.randn(2,3), columns=['b','d','a'])

df1

,a,b,c,d
0,1.365927,1.263798,-0.535745,-1.026820
1,-1.219033,-2.379581,1.289412,0.144180
2,0.195487,0.829485,-1.053597,-0.471227


In [76]:
df2

,b,d,a
0,-1.956031,0.601313,-0.932362
1,0.739163,0.393950,1.183197


In [77]:
pd.concat([df1,df2],ignore_index=True)

,a,b,c,d
0,1.365927,1.263798,-0.535745,-1.026820
1,-1.219033,-2.379581,1.289412,0.144180
2,0.195487,0.829485,-1.053597,-0.471227
3,-0.932362,-1.956031,NaN,0.601313
4,1.183197,0.739163,NaN,0.393950


### Combinando dados com sobreposição

In [78]:
a = pd.Series([np.nan,2.5,0.0,3.5,4.5,np.nan], index= ['f','e','d','c','b','a'])

b = pd.Series([0.,np.nan,2.,np.nan,np.nan,5.],index=['a','b','c','d','e','f'])

a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [79]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [80]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [81]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

In [88]:
df1 = pd.DataFrame({'a':[1.,np.nan,5.,np.nan],'b':[np.nan,2.,np.nan,6.],'c':range(2,18,4)})

df2 = pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],'b':[np.nan,3.,4.,6.,8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [89]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [90]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reformatação e pivoteamento

### Reformatação com indexação hierárquica

In [94]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),index=pd.Index(['Ohio','Colorado'],name='State'),columns=pd.Index(['one','two','three'],name='number'))

data

number,one,two,three
State,,,
Ohio,0,1,2
Colorado,3,4,5


In [95]:
result = data.stack()

result

State     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [96]:
result.unstack()

number,one,two,three
State,,,
Ohio,0,1,2
Colorado,3,4,5


In [97]:
result.unstack(0)

State,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [99]:
result.unstack('State')

State,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [100]:
s1 = pd.Series([0,1,2,3],index=['a','b','c','d'])

s2 = pd.Series([4,5,6],index=['c','d','e'])

data2 = pd.concat([s1,s2],keys=['one','two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [101]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [102]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [103]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [104]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [105]:
df = pd.DataFrame({'left':result,'right':result+5},columns=pd.Index(['left','right'],name='side'))

df

side             left  right
State    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [107]:
df.unstack('State')

side   left          right         
State  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [109]:
df.unstack('State').stack('side')

State         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Fazendo pivotamento de um formato "longo" para um formato "largo"

In [110]:
data = pd.read_csv('pydata-book/examples/macrodata.csv')

data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [111]:
periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')

columns = pd.Index(['realgdp','infl','unemp'],name='item')

data = data.reindex(columns=columns)

data.index = periods.to_timestamp('D','end')

ldata = data.stack().reset_index().rename(columns={0:'value'})

ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [112]:
pivoted = ldata.pivot('date','item','value')

pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [114]:
ldata['value2'] = np.random.randn(len(ldata))

ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.604215
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.621082
2,1959-03-31 23:59:59.999999999,unemp,5.800,-1.332629
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.532809
4,1959-06-30 23:59:59.999999999,infl,2.340,1.278153
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.374513
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.511637
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.064956
8,1959-09-30 23:59:59.999999999,unemp,5.300,2.171312
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-1.823734


In [115]:
pivoted = ldata.pivot('date','item')

pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.621082 -0.604215   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.278153  0.532809   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.064956  0.511637   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.948982 -1.823734   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.258300 -0.467408   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.332629  
1959-06-30 23:59:59.999999999  0.374513  
1959-09-30 23:59:59.999999999  2.171312  
1959-12-31 23:59:59.999999999  0.952400  
1960-03-31 23:59:59.999999999 -0.955087

In [116]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [117]:
unstacked = ldata.set_index(['date','item']).unstack('item')

unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.621082 -0.604215   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.278153  0.532809   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.064956  0.511637   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.948982 -1.823734   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.258300 -0.467408   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.978922 -0.710779   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  1.134623 -0.541522   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.332629  
1959-06-30 23:59:59.999999999  0.374513  
1959-09-30 23:59:59.999999999  2.171312  
1959-12-31 23:59:59.999999999  0.952400  
1960-03-31 23:59:59.999999999 -0.955087  
1960-06-30 23:59:59.999999999  0.873221  
1960-09-30 23:59:59.999999999  0.976630

### Pivoteamento do formato "longo" para o formato "longo"

In [118]:
df = pd.DataFrame({'key':['foo','bar','baz'],'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [119]:
melted = pd.melt(df,['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [120]:
reshaped = melted.pivot('key','variable','value')

reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [121]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [122]:
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [123]:
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [124]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
